# Chapter 38: Turbopack Deep Dive

Turbopack represents the next generation of JavaScript bundling for Next.js, replacing Webpack with a Rust-based engine designed for incremental compilation at scale. While Webpack revolutionized the JavaScript ecosystem, its JavaScript-based architecture struggles with the massive dependency trees of modern applications. Turbopack leverages Rust's memory safety and parallelization capabilities to achieve 10x faster cold starts and near-instant Hot Module Replacement (HMR), fundamentally changing the development experience for large-scale Next.js applications.

By the end of this chapter, you'll master understanding Turbopack's incremental architecture, configuring advanced build options, migrating existing Webpack configurations, optimizing file system caching, troubleshooting compatibility issues, and benchmarking performance improvements in real-world applications.

## 38.1 Turbopack Fundamentals

Understand the architectural differences between Turbopack and traditional bundlers, and how to enable it in your Next.js application.

### Enabling Turbopack

```javascript
// next.config.js
/** @type {import('next').NextConfig} */
const nextConfig = {
  // Enable Turbopack for development (default in Next.js 15+)
  turbopack: {
    // Resolve aliases matching tsconfig paths
    resolveAlias: {
      '@/*': './src/*',
      '@components/*': './src/components/*',
      '@lib/*': './src/lib/*',
    },
    
    // Resolve extensions priority
    resolveExtensions: ['.tsx', '.ts', '.jsx', '.js', '.json'],
    
    // Module rules for specific file types
    rules: {
      '*.svg': {
        loaders: ['@svgr/webpack'],
        as: '*.js',
      },
    },
  },
  
  // Experimental features (use with caution)
  experimental: {
    // Enable turbo mode for builds (experimental)
    turbo: {
      // Tree shaking configuration
      treeShaking: true,
      
      // Module ID strategy for caching
      moduleIdStrategy: 'deterministic',
    },
  },
};

module.exports = nextConfig;
```

```json
// package.json scripts
{
  "scripts": {
    "dev": "next dev --turbo",  // Explicitly enable Turbopack
    "dev:webpack": "next dev",   // Fallback to Webpack for comparison
    "build": "next build",
    "build:turbo": "next build --turbo" // Experimental production builds
  }
}
```

### Understanding Incremental Compilation

```typescript
// lib/build/incremental.ts
// Turbopack tracks dependencies at the module level, enabling granular updates

/**
 * Turbopack's Architecture Advantages:
 * 
 * 1. Fine-grained Caching: Only recompiles changed modules and their direct dependents
 * 2. Parallel Processing: Rust's thread safety allows maximum CPU utilization
 * 3. Persistent Caching: Disk-based cache survives restart (unlike Webpack's memory cache)
 * 4. Lazy Evaluation: Only processes modules when imported (tree shaking by default)
 */

// Example: In a large app, changing a utility function only rebuilds:
// - The utility module
// - Direct importers (5-10 files)
// NOT the entire dependency tree (Webpack behavior)

export function demonstrateIncremental() {
  // Before (Webpack): Change in utils/date.ts → 30s rebuild
  // After (Turbopack): Change in utils/date.ts → 50ms rebuild
  
  return {
    coldStart: '3-5x faster',
    hmrUpdate: '10-20x faster',
    memoryUsage: '40% lower',
  };
}
```

## 38.2 Webpack to Turbopack Migration

Migrate existing Webpack configurations and custom loaders to Turbopack's Rust-based ecosystem.

### Configuration Migration Guide

```javascript
// webpack.config.js (Old - for reference)
module.exports = {
  module: {
    rules: [
      {
        test: /\.svg$/,
        use: ['@svgr/webpack'],
      },
      {
        test: /\.md$/,
        use: 'raw-loader',
      },
    ],
  },
  resolve: {
    alias: {
      '@components': path.resolve(__dirname, 'src/components'),
    },
    fallback: {
      crypto: false,
      stream: false,
    },
  },
  plugins: [
    new webpack.DefinePlugin({
      __VERSION__: JSON.stringify(packageJson.version),
    }),
  ],
};

// next.config.js (New - Turbopack equivalent)
const nextConfig = {
  turbopack: {
    // SVG handling via loaders
    rules: {
      '*.svg': {
        loaders: ['@svgr/webpack'],
        as: '*.tsx', // Output as TSX for type safety
      },
      '*.md': {
        loaders: ['raw-loader'],
        as: '*.js',
      },
    },
    
    // Path aliases (reads tsconfig.json automatically, but can override)
    resolveAlias: {
      '@components': './src/components',
    },
    
    // Polyfill fallbacks (limited support compared to Webpack)
    resolveFallback: {
      // Turbopack supports fewer Node.js polyfills
      // Prefer native browser APIs or import polyfills explicitly
    },
  },
  
  // Environment variables (replaces DefinePlugin)
  env: {
    VERSION: process.env.npm_package_version,
  },
};
```

### Custom Loaders and Transforms

```typescript
// lib/build/custom-transform.ts
// Turbopack uses Rust-based transforms instead of JavaScript loaders

/**
 * Migration Strategy:
 * 
 * 1. Native Turbopack support: CSS, TypeScript, JSX, JSON, Assets
 * 2. Webpack loader compatibility: Use 'loaders' array for simple transforms
 * 3. Complex loaders: Move to build scripts or PostCSS plugins
 */

// For unsupported loaders, use next.config.js workaround:
const nextConfig = {
  webpack: (config, { isServer }) => {
    // Keep Webpack for specific production optimizations
    // while using Turbopack for development
    if (!process.env.TURBOPACK) {
      config.module.rules.push({
        test: /\.custom$/,
        use: 'custom-loader',
      });
    }
    return config;
  },
  
  turbopack: {
    // For development, use alternative approaches:
    // 1. Pre-process files in separate script
    // 2. Use PostCSS for CSS transforms
    // 3. Use SWC plugins for code transforms
  },
};
```

### PostCSS Integration

```javascript
// postcss.config.js
// Turbopack has first-class PostCSS support
module.exports = {
  plugins: {
    tailwindcss: {},
    autoprefixer: {},
    
    // Custom PostCSS plugins work without Webpack
    'postcss-import': {},
    'postcss-preset-env': {
      features: { 'custom-properties': false },
    },
    
    // CSS optimization for production
    ...(process.env.NODE_ENV === 'production' ? {
      cssnano: { preset: 'default' },
    } : {}),
  },
};

// next.config.js - Ensure PostCSS is processed
const nextConfig = {
  turbopack: {
    // PostCSS is automatically detected and used
    // No manual configuration needed
  },
};
```

## 38.3 Performance Optimization

Leverage Turbopack's caching and parallelization for maximum build speed.

### File System Caching Strategy

```javascript
// next.config.js
const nextConfig = {
  turbopack: {
    // Cache configuration (Next.js 15+)
    experimental: {
      // Turbopack stores cache in .next/cache/turbopack
      // This persists across restarts unlike Webpack's memory cache
      
      // Garbage collection for old cache entries
      cacheGcInterval: 1000 * 60 * 60, // 1 hour
      
      // Maximum cache size (MB)
      maxCacheSize: 1024,
    },
    
    // Module resolution caching
    resolve: {
      // Cache module resolution results
      preferRelative: true,
    },
  },
};
```

```bash
# Cache management commands

# Clear Turbopack cache (useful when encountering weird behavior)
rm -rf .next/cache/turbopack

# Or use Next.js built-in clean
npx next clean

# Environment variable to disable cache (debugging)
NEXT_TELEMETRY_DISABLED=1 npx next dev --turbo
```

### Memory and CPU Optimization

```javascript
// next.config.js
const nextConfig = {
  experimental: {
    turbo: {
      // Memory allocation for Rust-based compilation
      // Default is usually optimal, but adjust for large monorepos
      
      // Number of concurrent workers (defaults to CPU count)
      workers: 4,
      
      // Memory limit per worker (MB)
      memoryLimit: 4096,
      
      // Enable/Disable parallelization of specific tasks
      parallelCodeProcessing: true,
      parallelAssetProcessing: true,
    },
  },
  
  // For monorepos: Define module boundaries
  transpilePackages: [
    'ui-components',
    'shared-utils',
    '@company/design-system',
  ],
};
```

## 38.4 Troubleshooting and Compatibility

Handle common migration issues and compatibility gaps between Webpack and Turbopack.

### Common Issues and Solutions

```typescript
// lib/build/compatibility.ts
/**
 * Common Turbopack Issues & Fixes:
 * 
 * 1. Loader Not Supported
 *    Error: "loader X is not supported in Turbopack"
 *    Fix: Move to PostCSS, SWC plugin, or pre-build step
 * 
 * 2. Polyfill Errors
 *    Error: "Module not found: crypto"
 *    Fix: Install specific polyfill: npm install crypto-browserify
 *         Then import explicitly in code, don't rely on automatic polyfills
 * 
 * 3. Environment Variables
 *    Issue: process.env.XYZ undefined
 *    Fix: Ensure env vars are defined in next.config.js env:{} or .env files
 * 
 * 4. HMR Not Working
 *    Fix: Check for circular dependencies (Turbopack is stricter)
 *         Ensure components export properly (no anonymous default exports)
 */

// Debugging configuration
const debugConfig = {
  turbopack: {
    // Enable verbose logging
    logLevel: 'verbose', // 'none' | 'error' | 'warn' | 'info' | 'verbose'
    
    // Show dependency graph
    printDependencyGraph: process.env.DEBUG_TURBO === 'true',
  },
};
```

### Fallback to Webpack Strategy

```javascript
// next.config.js - Hybrid approach during migration
const nextConfig = {
  // Use Turbopack for dev, Webpack for production builds
  // (until Turbopack production builds are stable)
  
  webpack: (config, { dev, isServer }) => {
    // Only use Webpack customizations for production
    if (!dev) {
      config.module.rules.push({
        test: /\.legacy$/,
        use: 'legacy-loader',
      });
    }
    return config;
  },
  
  turbopack: {
    // Development-only configurations
    // Production uses Webpack automatically in Next.js 14
    // Next.js 15+ uses Turbopack for production (experimental)
  },
};

// Conditional script execution
// package.json
{
  "scripts": {
    "dev": "next dev --turbo",
    "dev:safe": "next dev", // Webpack fallback if Turbopack breaks
    "build": "next build",  // Uses Webpack (stable) or Turbopack (experimental --turbo flag)
    "build:webpack": "NODE_OPTIONS='--no-turbo' next build"
  }
}
```

### Environment Variable Debugging

```bash
# .env.local - Turbopack specific debug flags

# Enable detailed tracing
NEXT_PRIVATE_TRACE=1

# Debug module resolution
NEXT_PRIVATE_TURBO_LOG=trace

# Profile performance
NEXT_TURBOPACK_PROFILE=1

# Disable cache for clean testing
NEXT_PRIVATE_TURBO_DISK_CACHE=0
```

## 38.5 Advanced Configuration Patterns

Optimize Turbopack for monorepos, micro-frontends, and edge cases.

### Monorepo Configuration

```javascript
// next.config.js in app package of monorepo
const path = require('path');

const nextConfig = {
  // Turbopack handles monorepo packages efficiently
  transpilePackages: [
    '@repo/ui',
    '@repo/utils',
    '@repo/types',
  ],
  
  turbopack: {
    // Resolve workspace packages
    resolveAlias: {
      '@repo/*': path.join(__dirname, '../../packages/*/src'),
    },
    
    // Watch workspace packages for changes
    experimental: {
      watchOptions: {
        // Watch parent directories for changes in monorepo
        ignored: ['**/node_modules/**', '**/.git/**'],
      },
    },
  },
  
  // Ensure changes in workspace packages trigger HMR
  onDemandEntries: {
    // Keep pages in memory longer during development
    maxInactiveAge: 60 * 60 * 1000,
    pagesBufferLength: 5,
  },
};
```

### Import Assertions and JSON Modules

```typescript
// Turbopack supports standard import assertions
// app/config/page.tsx

import siteConfig from '@/config/site.json' assert { type: 'json' };
// Or for TypeScript 4.5+ style:
import type { Config } from '@/types';
import configData from '@/config/data.json' with { type: 'json' };

export default function ConfigPage() {
  // Type-safe JSON imports work natively in Turbopack
  const config: Config = configData;
  
  return (
    <div>
      <h1>{siteConfig.title}</h1>
      <pre>{JSON.stringify(config, null, 2)}</pre>
    </div>
  );
}
```

## Key Takeaways from Chapter 38

1. **Enabling Turbopack**: Use `next dev --turbo` flag or configure `turbopack: {}` in `next.config.js`. Turbopack is the default dev server in Next.js 15+, but can be explicitly enabled in earlier versions for testing.

2. **Configuration Migration**: Move Webpack `resolve.alias` to `turbopack.resolveAlias`, and loader rules to `turbopack.rules` with `loaders` arrays. Note that complex Webpack plugins may not have Turbopack equivalents yet—maintain dual Webpack/Turbopack configs during transition.

3. **Performance Characteristics**: Turbopack offers 10x faster cold starts than Webpack by parallelizing work across CPU cores using Rust. HMR updates complete in milliseconds rather than seconds because only changed modules and their direct dependents are recompiled, not the entire dependency graph.

4. **Caching Strategy**: Turbopack uses persistent disk caching in `.next/cache/turbopack` that survives restarts, unlike Webpack's memory-based cache. Clear this directory when encountering stale cache issues or weird behavior after package updates.

5. **Compatibility Limitations**: Turbopack supports fewer Node.js polyfills than Webpack. Explicitly install and import browser polyfills (e.g., `crypto-browserify`) rather than relying on `resolve.fallback` configuration. Some Webpack loaders must be replaced with PostCSS plugins or SWC transforms.

6. **Monorepo Optimization**: Use `transpilePackages` to enable HMR for workspace dependencies without manual building. Turbopack watches symlinked packages automatically, providing instant updates when shared components change.

7. **Debugging**: Set `NEXT_PRIVATE_TURBO_LOG=trace` for verbose output when troubleshooting module resolution issues. Use `next dev` (without `--turbo`) as a fallback when encountering unsupported loader errors, maintaining parallel configurations until full migration is possible.

## Coming Up Next

**Chapter 39: Build Adapters API**

With your development and build tooling optimized with Turbopack, it's time to explore how Next.js adapts to different deployment platforms. In Chapter 39, we'll dive into the Build Adapters API for custom output formats, platform-specific optimizations, edge runtime adapters, and creating custom deployment targets. You'll learn how Next.js transforms your application for serverless functions, static hosting, and edge networks.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='37. development_tooling.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='39. build_adapters_api.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
